In [0]:
import numpy as np
import os 
import cv2 as cv
import keras
import imageio
import keras
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from google.colab import files
from tqdm import tqdm
from keras import models , regularizers, layers, losses
from random import shuffle
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import load_img,img_to_array

np.random.seed(seed=0) #Constant Seed at 0

def shuffleData(Data,Labels): #This is for shuffeling data
    m=Data.shape[0]
    permutation=list(np.random.permutation(m))
    Data=Data[permutation, :, :]
    Labels=Labels[permutation]
    return Data, Labels
  
def PlotModelGraph(Model_History):  #This is for plotting the model after its done
    print(Model_History.history.keys())
    #Plot the accuracy
    plt.plot(Model_History.history['acc'])
    plt.plot(Model_History.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()
    #Plot the loss
    plt.plot(Model_History.history['loss'])
    plt.plot(Model_History.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='best')
    plt.show()
    return

get_ipython().system_raw("unrar x MAHDBase_TrainingSet.rar")  #Extracting the .rar files
get_ipython().system_raw("unrar x MAHDBase_TestingSet.rar")   #Extracting the .rar files


Training_Data = []
Training_Labels = []
Test_Data = []
Test_Labels = []

#----------------------------------Reading and labelling the data-----------------------------
for i in tqdm(range(1, len(os.listdir('./MAHDBase_TrainingSet')) + 1)):
  if(i<10):
    for j in range((len(os.listdir('./MAHDBase_TrainingSet' + '/Part0' + str(i))))):

      if '.bmp' in str(os.listdir('./MAHDBase_TrainingSet' + '/Part0' + str(i))[j]):
        path = './MAHDBase_TrainingSet' + '/Part0' + str(i)
        image = imageio.imread(os.path.join(path, os.listdir('./MAHDBase_TrainingSet' + '/Part0' + str(i))[j]), as_gray=True)
        Training_Data.append(image)
        for k in range(10):
          if 'digit' + str(k) in str(os.listdir('./MAHDBase_TrainingSet' + '/Part0' + str(i))[j]):
            Training_Labels.append(k)
            break
            
  else:
    for l in range((len(os.listdir('./MAHDBase_TrainingSet' + '/Part' + str(i))))):
       if '.bmp' in str(os.listdir('./MAHDBase_TrainingSet' + '/Part' + str(i))[l]):
          path = './MAHDBase_TrainingSet' + '/Part' + str(i)
          image = imageio.imread(os.path.join(path, os.listdir('./MAHDBase_TrainingSet' + '/Part' + str(i))[l]), as_gray=True)
          Training_Data.append(image)
          for k in range(10):
            if 'digit' + str(k) in str(os.listdir('./MAHDBase_TrainingSet' + '/Part' + str(i))[j]):
              Training_Labels.append(k)
              break
          
          

for i in tqdm(range(1, len(os.listdir('./MAHDBase_TestingSet')) + 1)):
  for j in range((len(os.listdir('./MAHDBase_TestingSet' + '/Part0' + str(i))))):
    if '.bmp' in str(os.listdir('./MAHDBase_TestingSet' + '/Part0' + str(i))[j]):
      path = './MAHDBase_TestingSet' + '/Part0' + str(i)
      image = imageio.imread(os.path.join(path, os.listdir('./MAHDBase_TestingSet' + '/Part0' + str(i))[j]), as_gray=True)
      Test_Data.append(image)
      for k in range(10):
            if 'digit' + str(k) in str(os.listdir('./MAHDBase_TestingSet' + '/Part0' + str(i))[j]):
              Test_Labels.append(k)
              break
#---------------------------------------------------------------------------------------------------      

Training_Data = np.array(Training_Data)
Training_Labels = np.array(Training_Labels)
Test_Data = np.array(Test_Data)
Test_Labels = np.array(Test_Labels)

Training_Data = Training_Data.reshape(60000, 28, 28, 1)  #Making sure shapes are correct
Test_Data = Test_Data.reshape(10000, 28, 28, 1)          #Making sure shapes are correct

Shuffled_Training_Data, Shuffled_Training_Labels = shuffleData(Training_Data, Training_Labels)      #Shuffeling data
Shuffled_Test_Data, Shuffled_Test_Labels = shuffleData(Test_Data, Test_Labels)                      #Shuffeling data

Training_Labels = to_categorical(Shuffled_Training_Labels)
Test_Labels = to_categorical(Shuffled_Test_Labels)
#---------------------------------Model--------------------------------------------------------
model = Sequential()
model.add(layers.Conv2D(12, kernel_size= 5, strides=(1, 1), activation='tanh', input_shape=(28,28,1), padding="same"))
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
model.add(layers.Conv2D(32, kernel_size= 5, strides=(1, 1), activation='tanh', padding='valid'))
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))
model.add(layers.Conv2D(120, kernel_size= 5, strides=(1, 1), activation='tanh', padding='valid'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
#----------------------------------------------------------------------------------------------
#Optimizers experimenting with
Adam_optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
SGD = keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
RMSP = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(loss= losses.mean_squared_error, optimizer= RMSP, metrics=["accuracy"])    #Compiling model
History = model.fit(Shuffled_Training_Data, Training_Labels, epochs=100, batch_size=128, validation_data=(Shuffled_Test_Data, Test_Labels), verbose=1)   #Training the model
PlotModelGraph(History) #Plotting

Predicted_Classes = model.predict_classes(Shuffled_Test_Data, verbose=0)   #Getting scores

precision, recall, fscore, support = score(Shuffled_Test_Labels, Predicted_Classes)

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))